# PLanet tutorial 
Welcome! This is a tutorial for PLanet. We will walk through examples defining
experimental designs as a program. You will learn some of Planet's operations
and observe the output of PLanet programs. Let's begin!


In [ ]:
# First, import the PLanet library
from planet import *

## Step 1: Defining Experiment Variables
Before constructing a design, we need to know which to include in the experiment. $ExperimentVariables$ in PLanet represent independent and control
variables. In the following example, Interface is the name of the variable, and there
are three interface conditions (options): baseline, VR, and AR.

In [ ]:

interface = ExperimentVariable("interface", options=["baseline", "VR", "AR"])

We also need to identify the experimental units in the study. Often,
experimental units are participants. We will assign variable conditions to these
units. Let's sample 12 participants:

In [ ]:
# Replace n with the number of units in our study (12)
participants = Units(12)

## Step 2: Creating a Design
Now that we've defined our experiment variables, we can start constructing a design. The most
basic design in PLanet is a between-subjects design with one experiment
variable. First, let's instantiate a *Design* that includes interface as a
between-subjects variable:


In [ ]:
# a between subjects design
design = (
    Design()
        .between_subjects(interface) # include interface as an argument 
)

Between-subjects designs assign one condition to each *unit*. By default, the
conditions are randomly assigned to units. Run the cell below and observe the output:

In [ ]:
# Our first assignment procedure
print(assign(participants, design))

## Step 3: Within-subjects Designs
Now, we can explore more complicated designs. A variable is within-subjects in
an experiment if a participant is assigned two or more of it's conditions. Let's
construct a new design with interface as a within-subjects variable:

In [ ]:
interface_design = (
    Design()
        .within_subjects(interface) # include interface as argument
)

The following output shows the result of assigning conditions using a
within-subjects design. Each unit is assigned all three conditions in a
different order.

In [ ]:
print(assign(participants, interface_design))

Theoretically, there is a chance
that we assign every unit the same order! Counterbalancing prevents this by
enforcing that every condition appears in every position of an order an equal number of times:

In [ ]:
interface_design = (
    interface_design
        .counterbalance(interface) # include interface as an argument
)

In [ ]:
print(assign(participants, interface_design))

PLanet first constructs all viable plans, and then maps each plan the
experimental unit. 

By default, the design assigns every condition to each unit. We can define a
design where we assign two conditions to each unit and there are three
conditions: 

In [ ]:
interface_design = (
    interface_design
          .num_trials(2)
)

print(assign(participants, interface_design))


## Step 4: Latin square Designs
This introduces a new operation that allows you to construct Latin
squares. Latin squares are a particular type of counterbalanced design, where
every condition appears in every position of an order once. So, for
counterbalanced designs with three conditions, there are only three possible
orders. We will start with a regular counterbalanced design: 

In [ ]:
task = ExperimentVariable(
    name = "task",
    options=["run", "walk", "sprint"]
)

task_design = (
    Design()
    .within_subjects(task)
    .counterbalance(task, )
)


This defines fully-counterbalanced design. Fully-counterbalanced designs are
counterbalanced designs with the *maximum* number of plans, resulting in all
possible permutations. Latin sqaures are counterbalanced designs with the
*minimum* number of rows. Each condition should appear once in every position of
an order.
The number of orders is the same as the number of conditons. We can constrain the design using the
*limit_plans* operation to create a Latin square:

In [ ]:
task_design = (
    task_design
    .limit_groups(len(task))
)

When we run the assignment procedure, PLanet constructs three plans, ensuring
that the task conditions are counterbalanced:

In [ ]:
print(assign(participants, task_design))

## Step 5: Composing Designs
Great! We created two designs that specify assignment procedures for different
variable. We can also compose these procedures into one design using *nest* and
*cross* 

### Nesting
Nest creates a new design based on the constraints of two subdesigns (i.e.,
$task\_design$ and $interface\_design$). The possible plans specified by the
*inner* design are nested within each trial of the *outer* design. 


![Nesting](nest.png)

In [ ]:
design = nest(inner=interface_design, outer=task_design)

Let's examine the possible plans:

In [ ]:
print(assign(participants, design))

### Cross
Cross creates a new design based on the constraints of two subdesigns (i.e.,
$task\_design$ and $interface\_design$). The composed design overlays every plan
from the first design with every plan of the second design. Each subdesign must
have the same number of trials. 

![Cross](cross.png)

## Step 6: Replication
Sometimes, we want to assign the same experimental condition multiple times. We can replicate
conditions by nesting a non-empty design with an *empty* design. Empty designs
assign arbitrary conditions to users. To define an empty design, we do not
include any variables in the experiment. The number of trials will indicate the
number of replications after nesting the empty design. 

In [ ]:
block = (
    Design()
    .num_trials(2)
)

# NOTE: task_design is the latin sqaure we defined in Step 3. 
design = nest(inner=block, outer=task_design)
print(assign(participants, design))

This repeats each condition in-a-row ([a, b] -> [a, a, b, b]).
Alternatively, we could repeat the orders ([a, b] -> [a, b, a, b])

In [ ]:
block = (
    Design()
    .num_trials(2)
)

design = nest(inner=task_design, outer=block)
print(assign(participants, design))

## Step 6: $set\_rank$
Lastly, we invite you to explore the $set\_rank$ operation. *Ranks* determine the order
presedence of variable conditons. The default rank is 0 for all conditions of a
variable. Higher-ranked conditions come before all lower-ranked conditions in
within-subjects designs. For example, if we set the rank of baseline to 1, baseline
always
comes first. Run the code and observe the output: 

In [ ]:
task = ExperimentVariable(
    name = "task",
    options=["run", "walk", "sprint"]
)

participants = Units(4)

task_design = (
    Design()
    .within_subjects(task)
    .absolute_rank(task, "run", 1)
)

print(assign(participants, task_design))

Notice that walk and sprint can appear in any order, as long as they both come run. 

# Step 7: multifactfactorial designs
Multifactorial designs are expeirmental designs with more than one variable.
Step 5 systematically composed designs using *nest* and *cross*. We can also
define multifactorial variables by adding multiple variables to a design. We can
do this using multiple $within\_subjects$ and $between\_subjects$ expressions, or
with the $multifact$ operation.

### multifact 
multifact creates a new variable. The conditions of this variable are determined
by taking the cartesian product of the conditions of a set of variables. For
example, multifact([task, interface]) results in a new variable with the
following conditions: VR-run VR-walk, VR-sprint, AR-run AR-walk, AR-sprint,
baseline-run baseline-walk, baseline-sprint. 

In [ ]:
treatment = ExperimentVariable(
    name = "treatment",
    options = ["a", "b", "c"]
)

task = ExperimentVariable(
    name = "task",
    options = ["1", "2", "3"]
)


des = (
    Design()
        .within_subjects(multifact([treatment, task]))
        .counterbalance(multifact([treatment, task]))
        .limit_groups(9)
)

units = Units(12)

print(assign(units, des))

We can also add variables to a design and counterbalance them independently 

In [ ]:
task_design = (
    Design()
    .within_subjects(task)
    .within_subjects(interface)
    .counterbalance(task)
    .num_trials(2)
    .counterbalance(interface)
)
print(assign(participants, task_design))

## Conclusion
Congrats! You're done with the tutorial. Now that you're familiar with PLanet's core features, try authoring your own experimental design from scratch using the starter code! Don't worry if you didn't remember everything.
You can always reference the tutorial again, and we invite you to reference the
documentation for further information. 